In [1]:
import torch
from tqdm import tqdm
from model import seedformer_dim128
from torch.utils.data import Dataset,DataLoader
import open3d as o3d
import os
from pytorch3d.loss import chamfer_distance
import numpy as np
from torch.optim.lr_scheduler import StepLR
import utils.utils as utils

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


### Dataset and Dataloader

https://pytorch.org/tutorials/beginner/basics/data_tutorial.html
https://pytorch.org/tutorials/beginner/data_loading_tutorial.html

Below is the implementation of custom dataset/dataloader. Additionally there is a collate function. The number of points in eaach pointcloud is 
different. In order to make them equal collate function is defined.

In [2]:
class RacingDataset(Dataset):
    def __init__(self,root_dir):#4731
        self.root_dir = root_dir
        self.file_list = os.listdir(root_dir)
        self.filter_file_list = self.filter_list()
       # self.target_points = target_points

    def __len__(self):
        return len(self.filter_file_list)

    def __getitem__(self,index):
        pcd_path = os.path.join(self.root_dir,self.filter_file_list[index])
        pcd = o3d.io.read_point_cloud(pcd_path)

        points = torch.tensor(pcd.points, dtype=torch.float32)

        return points,pcd_path

   
    def filter_list(self):
        '''
        Filter the inputs so that only pcds with more than 50 points are included in the training
        :return:
        '''
        filtered_list=[]
        for filename in self.file_list:
            pcd = o3d.io.read_point_cloud(os.path.join(self.root_dir,filename))
            points = torch.tensor(pcd.points, dtype=torch.float32)
            if len(points)>=0:
                filtered_list.append(filename)
        return filtered_list
   


In [3]:

# Load the largest point cloud
pcd_pad = o3d.io.read_point_cloud("/home/omar/TUM/Data/cropped/sim/018840.pcd")
pcd_pad_tens = torch.tensor(pcd_pad.points, dtype=torch.float32)
#print(len(pcd_pad_tens))
# Create the dataset and dataloader
dataset = RacingDataset(root_dir="/home/omar/TUM/Data/cropped/real")
dataloader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=8,collate_fn=utils.collate_fn)

print(len(dataloader))

/tmp/ipykernel_7007/3415559981.py:3: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:261.)
  pcd_pad_tens = torch.tensor(pcd_pad.points, dtype=torch.float32)


5993


In [4]:
print(torch.__version__)
torch.cuda.is_available()

2.1.0+cu121


True

In [5]:
torch.device("cuda" if torch.cuda.is_available() else "cpu")

device(type='cuda')

# Training

In [6]:
torch.manual_seed(42)
#del model
model = seedformer_dim128(up_factors=[1, 2, 2,2])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)
epochs=1
optimizer=torch.optim.Adam(model.parameters(),lr=0.001, weight_decay=5e-4)
scheduler = StepLR(optimizer, step_size=5, gamma=0.1)  # Reduce LR by a factor of 0.1 every 30 epochs

# Initialize variables for checkpointing
#best_loss = float('inf')
# Define paths for checkpoint and log file
checkpoint_path = '/home/omar/TUM/Masterarbeit/Codes/training_2604v001.pth'
log_file = 'training_2604v001.pth.txt'

# Check if a checkpoint exists
if os.path.exists(checkpoint_path):
    # Load the checkpoint
    checkpoint = torch.load(checkpoint_path)
    start_epoch = checkpoint['epoch'] + 1
    best_loss = checkpoint['loss']
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
    print(f"Checkpoint loaded. Resuming training from epoch {start_epoch}. Best loss so far: {best_loss}")
else:
    start_epoch = 0
    best_loss = float('inf')
    print("No checkpoint found. Starting training from scratch.")

# Training loop
for epoch in range(start_epoch, epochs):
    running_loss = 0

    # Wrap the DataLoader with tqdm to track progress
    with tqdm(enumerate(dataloader, 0), total=len(dataloader), desc=f'Epoch {epoch + 1}/{epochs}', unit='batch') as pbar:
        for i, data in pbar:
            inputs, labels = data
            inputs = inputs.to(device)  # Move data to GPU if available

            optimizer.zero_grad()
            outputs = model(inputs)
            losses = []

            for input_pc, output_pc in zip(inputs, outputs[-1]):
                # Calculate Chamfer Distance loss using pytorch3d.loss.chamfer_distance
                loss, _ = chamfer_distance(input_pc.unsqueeze(0), output_pc.unsqueeze(0))
                #loss_total, loss, gts = utils.get_loss(outputs, inputs, inputs, sqrt=False)
                #print("loss_total " ,loss_total)
                #print("losses " ,loss)
                #print("gts " , gts)
                losses.append(loss)

            #loss = torch.mean(torch.stack(losses))
            loss.backward()
            optimizer.step()

            running_loss += loss
            pbar.set_postfix(loss=running_loss / (i + 1))  # Update tqdm progress bar with the current loss

        scheduler.step()  # Step the learning rate scheduler

    # Save checkpoint if current loss is the best seen so far
    if running_loss < best_loss:
        best_loss = running_loss
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'loss': running_loss
        }, checkpoint_path)
 
    # Log the epoch loss in the log file
    with open(log_file, 'a') as f:
        f.write(f'Epoch {epoch + 1} Loss: {running_loss / len(dataloader)}\n')

print('Finished Training')

cuda
Checkpoint loaded. Resuming training from epoch 8. Best loss so far: 0.019063685089349747
Finished Training


 ## Save The Data


In [7]:
#del model
#import gc         # garbage collect library
#gc.collect()
#torch.cuda.empty_cache() 

In [9]:
simulation_dataset = RacingDataset(root_dir="/home/omar/TUM/Data/cropped/sim")
simulation_dataloader = DataLoader(simulation_dataset, batch_size=1, shuffle=True, num_workers=8,collate_fn=utils.collate_fn)
utils.apply_and_save_res(dataset=simulation_dataset,dataloader=simulation_dataloader,model=model,savedir="/home/omar/TUM/Data/reconstructed_cropped/sim_2604v001")

100%|██████████| 5997/5997 [10:04<00:00,  9.92point cloud/s]


In [11]:
pcd=o3d.io.read_point_cloud("/home/omar/TUM/Data/reconstructed_cropped/sim_2604v001/027075.pcd")
#("/home/omar/TUM/data_MA/m1695833/Sim2RealDistributionAlignedDataset/sim/data/pcl/000035.pcd")
print(len(pcd.points))
#pcl=np.load("/home/omar/TUM/Data/reconstructed_cropped/sim_0204/020100.pcd")
#pcd = o3d.geometry.PointCloud()
#pcd.points = o3d.utility.Vector3dVector(pcl)
##box=["Car" ,0.0, 0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,1.18, 1.9 ,4.88 ,1.56 ,6.55 ,1.01 ,-0.0]
box=["Car" ,0.0, 0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,1.18, 1.9 ,4.88 ,26.11 ,-5.9 ,0.59 ,0.02]
#np.set_printoptions(threshold=np.inf)
#print(np.asarray(pcd.points))
#pcd_cut=crop_invert_stitch(pcl,box)
#pcd=pcd_cut+pcd_car_recons
utils.visualize(pcl=pcd,bbox_coordinates=box)

4096


## "Stitch" the data

In [12]:
reconstructed_car_path="/home/omar/TUM/Data/reconstructed_cropped/sim_2604v001/"
original_sim_path="/home/omar/TUM/data_MA/m1695833/Sim2RealDistributionAlignedDataset/sim/data/pcl/"
bbox_path="/home/omar/TUM/data_MA/m1695833/Sim2RealDistributionAlignedDataset/sim/data/label"
#reconstructed_scene_path="/home/omar/TUM/Data/reconstructed_scene/sim/"
#v002 create oints correctly
npy_final_path="/home/omar/TUM/Data/SeedFormer_2602_npy/reconstructed_2604v001/points/"
file_car_list=[]
for filename_car in os.listdir(reconstructed_car_path):
    file_car_list.append(filename_car)
    txt=filename_car.replace('.pcd',".txt")
    bbox = (open(bbox_path+"/"+txt, "r")).read().split()
    original_pcd = o3d.io.read_point_cloud(original_sim_path+filename_car)
    filename_=filename_car.split(".")[0]
    #if  (int)(filename_)<=24995:#<>
        
        #print(filename_)
    #np_array=np.asarray(original_pcd.points)#.pcd
    #arr2d=np.asarray(pcd_sim.points)
    #x=np_array.copy()[:,0]
    #np_array[:,0]=np_array[:,2]
    #np_array[:,2]=x
   # np.save(npy_final_path+filename_,np_array)#.npy

    reconstructed_car= o3d.io.read_point_cloud(reconstructed_car_path +"/"+ filename_car)
    crop_invert=utils.crop_invert_stitch(original_pcd,reconstructed_car, bbox)
    #o3d.io.write_point_cloud(reconstructed_scene_path+"/"+filename_car,crop_invert)
    np_array=np.asarray(crop_invert.points)
    np.save(npy_final_path+filename_,np_array)
#

## Create and save the new dataset

In [ ]:
import os
import open3d as o3d
import numpy as np

# Path to the folder containing PCD files
pcd_folder_path = '/home/omar/TUM/Data/reconstructed_cropped/sim_0404_30v004/'

# List all PCD files in the folder
file_list = [f for f in os.listdir(pcd_folder_path) if f.endswith('.pcd')]

# Iterate through each PCD file
for file_name in file_list:
    # Read the point cloud
    pcd = o3d.io.read_point_cloud(os.path.join(pcd_folder_path, file_name))
    
    # Get the numpy array of points
    points = np.asarray(pcd.points)
    
    # Find the indices of non-padded points (non-zero points along the first axis)
    non_padded_indices = np.where(np.any(points != 0, axis=1))[0]
    
    # Extract non-padded points
    non_padded_points = points[non_padded_indices]
    
    # Update the point cloud with non-padded points
    pcd.points = o3d.utility.Vector3dVector(non_padded_points)
    print(len(non_padded_points))  # Print the number of non-padded points
    print(non_padded_points[:5])   # Print the first 5 non-padded points

    # Save the updated point cloud back to file
    o3d.io.write_point_cloud(os.path.join(pcd_folder_path, file_name), pcd)


# Correcting Labels


In [23]:
labels_folder="/home/omar/TUM/data_MA/m1695833/Sim2RealDistributionAlignedDataset/sim/data/label/"
save_folder="/home/omar/TUM/Data/SeedFormer_2602_npy/sim/labels/"
for label in os.listdir(labels_folder):
    file = open(labels_folder+label)
    bbox=file.read()
    bbox_correct=utils.correct_bbox_label(bbox)
    file.close()
    file_write=open(save_folder+label,"w+")
    file_write.write(" ".join(bbox_correct))
    file_write.close()
    print(bbox_correct)
    #print(items)

['86.13', '2.08', '0.59', '4.88', '1.9', '1.18', '0.01', 'Car']
['-36.92', '-5.76', '0.59', '4.88', '1.9', '1.18', '-0.01', 'Car']
['42.03', '0.86', '0.59', '4.88', '1.9', '1.18', '-0.0', 'Car']
['-0.46', '-5.79', '0.6', '4.88', '1.9', '1.18', '-0.01', 'Car']
['93.19', '19.5', '4.5', '4.88', '1.9', '1.18', '0.4', 'Car']
['-44.16', '3.35', '0.95', '4.88', '1.9', '1.18', '-0.11', 'Car']
['75.86', '4.29', '0.91', '4.88', '1.9', '1.18', '0.08', 'Car']
['-34.94', '-3.96', '0.76', '4.88', '1.9', '1.18', '-0.08', 'Car']
['-24.56', '-3.99', '0.96', '4.88', '1.9', '1.18', '-0.12', 'Car']
['59.57', '8.24', '2.19', '4.88', '1.9', '1.18', '0.27', 'Car']
['44.82', '4.99', '1.61', '4.88', '1.9', '1.18', '0.2', 'Car']
['-64.58', '5.08', '1.05', '4.88', '1.9', '1.18', '-0.14', 'Car']
['-30.6', '-4.01', '0.82', '4.88', '1.9', '1.18', '-0.12', 'Car']
['38.49', '6.41', '1.3', '4.88', '1.9', '1.18', '0.22', 'Car']
['2.69', '-5.91', '0.59', '4.88', '1.9', '1.18', '0.01', 'Car']
['98.06', '3.84', '1.83', '4

In [ ]:
#pcl=np.load("/home/omar/TUM/Data/SeedFormer_2602_npy/reconstructed/points/018840.npy")
pcd=o3d.io.read_point_cloud("/home/omar/TUM/Data/cropped/real/018840.pcd")
print(len(pcd.points))
#pcd = o3d.geometry.PointCloud()
#pcd.points = o3d.utility.Vector3dVector(pcl)
#box=["Car" ,0.0, 0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,1.18, 1.9 ,4.88 ,1.56 ,6.55 ,1.01 ,-0.0]
box=["Car" ,0.0, 0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,1.18, 1.9 ,4.88 ,81.49 ,2.12,0.76 ,0.06]

#pcd_cut=crop_invert_stitch(pcl,box)
#pcd=pcd_cut+pcd_car_recons
utils.visualize(pcl=pcd,bbox_coordinates=box)
#utils.visualize(pcl=pcd_sim)

In [14]:
pcd_test=o3d.io.read_point_cloud("/home/omar/TUM/Data/cropped/real_512/015385.pcd")
#/home/omar/TUM/Data/SeedFormer_2602_npy/real/labels/014165.txt
#['0.46', '6.27', '0.54', '4.88', '1.9', '1.18', '-0.02', 'Car']

utils.visualize(pcd_test,box)


In [7]:
down_pcd=o3d.geometry.PointCloud.farthest_point_down_sample(pcd_test,len(pcd_test.points)//7)
box=["Car" ,0.0, 0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,1.18, 1.9 ,4.88 ,81.49 ,2.12,0.76 ,0.06]

utils.visualize(down_pcd,box)
print(len(pcd_test.points))

print(len(down_pcd.points))

2972
424


In [12]:
import os
import shutil
from open3d import io

# Define input and output directories
pcd_folder = "/home/omar/TUM/Data/cropped/real"
label_folder = "/home/omar/TUM/Data/SeedFormer_2602_npy/real/labels"
output_pcd_folder = "/home/omar/TUM/Data/cropped/real_512"
output_label_folder = "/home/omar/TUM/Data/cropped/real_512_labels"
output_txt_file = "/home/omar/TUM/Data/cropped/real_512_images.txt"

# Create output directories if they don't exist
os.makedirs(output_pcd_folder, exist_ok=True)
os.makedirs(output_label_folder, exist_ok=True)

# Open text file for writing
with open(output_txt_file, "w") as txt_file:

    # Get the list of PCD files
    pcd_files = [f for f in os.listdir(pcd_folder) if f.endswith(".pcd")]

    # Iterate over each PCD file
    for pcd_file in pcd_files:
        # Load the point cloud
        pcd_path = os.path.join(pcd_folder, pcd_file)
        pcd = io.read_point_cloud(pcd_path)

        # Check if the point cloud has more than 512 points
        if len(pcd.points) > 512:
            # Save the point cloud to the output folder
            output_pcd_path = os.path.join(output_pcd_folder, pcd_file)
            io.write_point_cloud(output_pcd_path, pcd)

            # Copy the corresponding label file
            label_file = pcd_file.replace(".pcd", ".txt")
            label_path = os.path.join(label_folder, label_file)
            output_label_path = os.path.join(output_label_folder, label_file)
            shutil.copy(label_path, output_label_path)

            # Write the name of the saved PCD file to the text file
            txt_file.write(pcd_file + "\n")

            # Print the name of the file
            print("Saved:", pcd_file)

# Print message when done
print("File names written to:", output_txt_file)

Saved: 025325.pcd
Saved: 025365.pcd
Saved: 029990.pcd
Saved: 026985.pcd
Saved: 009160.pcd
Saved: 001960.pcd
Saved: 009190.pcd
Saved: 011000.pcd
Saved: 020900.pcd
Saved: 001895.pcd
Saved: 011880.pcd
Saved: 010995.pcd
Saved: 006205.pcd
Saved: 011870.pcd
Saved: 020890.pcd
Saved: 016515.pcd
Saved: 005505.pcd
Saved: 001900.pcd
Saved: 027075.pcd
Saved: 021525.pcd
Saved: 013335.pcd
Saved: 029335.pcd
Saved: 014195.pcd
Saved: 029935.pcd
Saved: 017955.pcd
Saved: 025340.pcd
Saved: 025350.pcd
Saved: 011865.pcd
Saved: 023485.pcd
Saved: 029270.pcd
Saved: 009195.pcd
Saved: 015365.pcd
Saved: 021535.pcd
Saved: 024620.pcd
Saved: 028595.pcd
Saved: 028650.pcd
Saved: 019485.pcd
Saved: 013365.pcd
Saved: 024610.pcd
Saved: 014130.pcd
Saved: 029965.pcd
Saved: 017160.pcd
Saved: 009175.pcd
Saved: 029340.pcd
Saved: 026265.pcd
Saved: 001725.pcd
Saved: 011015.pcd
Saved: 016490.pcd
Saved: 009105.pcd
Saved: 026290.pcd
Saved: 009090.pcd
Saved: 017175.pcd
Saved: 027055.pcd
Saved: 015375.pcd
Saved: 014175.pcd
Saved: 001

In [24]:
import os
import shutil

# Define paths
pcd_folder =  "/home/omar/TUM/Data/SeedFormer_2602_npy/real/labels/"
destination_folder = "/home/omar/TUM/Data/SeedFormer_2602_npy/real_512/labels/"
txt_file = "/home/omar/TUM/Data/SeedFormer_2602_npy/sim_512/ImageSets/val.txt"

# Create destination folder if it doesn't exist
os.makedirs(destination_folder, exist_ok=True)

# Read file names from the text file
with open(txt_file, "r") as file:
    file_names = file.read().splitlines()

# Iterate over each file name and copy corresponding PCD file
for file_name in file_names:
    source_path = os.path.join(pcd_folder, file_name+".txt")
    destination_path = os.path.join(destination_folder, file_name+".txt")
    
    # Check if the file exists before copying
    if os.path.exists(source_path):
        shutil.copyfile(source_path, destination_path)
        #print(f"Copied {file_name} to {destination_folder}")
    else:
        print(f"File {file_name} not found. Skipping...")
print("All files copied successfully!")


All files copied successfully!


In [25]:
import os

# Define folder path and list of file names
folder_path =  "/home/omar/TUM/Data/SeedFormer_2602_npy/real_512/points"
txt_file_path =  "/home/omar/TUM/Data/SeedFormer_2602_npy/sim_512/labels/"
import os

def get_files_in_folder(folder_path):
    """
    Get the list of files in a folder.
    """
    return set(os.listdir(folder_path))

def find_missing_files(folder1_path, folder2_path):
    """
    Compare two folders and find the missing files in folder2.
    """
    folder1_files = get_files_in_folder(folder1_path)
    folder2_files = get_files_in_folder(folder2_path)
    
    missing_files = folder1_files - folder2_files
    
    return missing_files

# Define the paths to the two folders
folder1_path = "/home/omar/TUM/Data/SeedFormer_2602_npy/real_512/points"
folder2_path = "/home/omar/TUM/Data/SeedFormer_2602_npy/sim_512/points"

# Find missing files in folder2 compared to folder1
missing_files = find_missing_files(folder1_path, folder2_path)

# Print the missing files
print("Missing files in folder2 compared to folder1:")
for file_name in missing_files:
    print(file_name)


Missing files in folder2 compared to folder1:


In [28]:
import os
import shutil

def copy_files_from_list(file_list_path, source_folder, destination_folder):
    """
    Copy files listed in a text file from the source folder to the destination folder.
    """
    with open(file_list_path, "r") as file:
        filenames = file.read().splitlines()

    for filename in filenames:
        filename=filename+'.txt'
        source_file = os.path.join(source_folder, filename)
        destination_file = os.path.join(destination_folder, filename)
        # Check if the file exists in the source folder
        if os.path.isfile(source_file):
            # Copy the file to the destination folder
            shutil.copy2(source_file, destination_file)
            print(f"File '{filename}' copied successfully.")
        else:
            print(f"File '{filename}' does not exist in the source folder.")

# Define the paths to the source and destination folders
source_folder = "/home/omar/TUM/Data/SeedFormer_2602_npy/real_512/labels"
destination_folder = "/home/omar/TUM/Data/SeedFormer_2602_npy/sim_512/labels"
file_list_path = "/home/omar/TUM/Data/SeedFormer_2602_npy/real_512/ImageSets/val.txt"

# Copy files listed in the text file to the destination folder
copy_files_from_list(file_list_path, source_folder, destination_folder)


File '020900.txt' copied successfully.
File '020890.txt' copied successfully.
File '021525.txt' copied successfully.
File '023485.txt' copied successfully.
File '021535.txt' copied successfully.
File '024620.txt' copied successfully.
File '024610.txt' copied successfully.
File '020930.txt' copied successfully.
File '024590.txt' copied successfully.
File '023540.txt' copied successfully.
File '020935.txt' copied successfully.
File '023565.txt' copied successfully.
File '024605.txt' copied successfully.
File '022700.txt' copied successfully.
File '021465.txt' copied successfully.
File '023495.txt' copied successfully.
File '021490.txt' copied successfully.
File '022670.txt' copied successfully.
File '024565.txt' copied successfully.
File '020960.txt' copied successfully.
File '023525.txt' copied successfully.
File '022680.txt' copied successfully.
File '024580.txt' copied successfully.
File '021485.txt' copied successfully.
File '023570.txt' copied successfully.
File '020895.txt' copied 

In [27]:
import os
import shutil

def replace_files(source_folder, destination_folder, file_list_path):
    """
    Replace files in the destination folder with the corresponding ones from the source folder based on the list of filenames.
    """
    # Read the list of filenames from the text file
    with open(file_list_path, "r") as file:
        filenames = file.read().splitlines()

    # Iterate over each filename and copy the corresponding file from the source to the destination folder
    for filename in filenames:
        filename=filename+'.txt'
        source_file = os.path.join(source_folder, filename)
        destination_file = os.path.join(destination_folder, filename)
        
        # Check if the file exists in the source folder
        if os.path.isfile(source_file):
            # Replace the file in the destination folder with the one from the source folder
            shutil.copy2(source_file, destination_file)
            print(f"File '{filename}' replaced successfully.")
        else:
            print(f"File '{filename}' does not exist in the source folder.")

# Define the paths to the source and destination folders, and the file list
file_list_path = "path/to/file_list.txt"
source_folder = "/home/omar/TUM/Data/SeedFormer_2602_npy/sim/labels"
destination_folder = "/home/omar/TUM/Data/SeedFormer_2602_npy/sim_512/labels"
file_list_path = "/home/omar/TUM/Data/SeedFormer_2602_npy/sim_512/ImageSets/train.txt"
# Replace files in the destination folder with the corresponding ones from the source folder
replace_files(source_folder, destination_folder, file_list_path)


File '025325.txt' replaced successfully.
File '025365.txt' replaced successfully.
File '029990.txt' replaced successfully.
File '026985.txt' replaced successfully.
File '009160.txt' replaced successfully.
File '001960.txt' replaced successfully.
File '009190.txt' replaced successfully.
File '011000.txt' replaced successfully.
File '001895.txt' replaced successfully.
File '011880.txt' replaced successfully.
File '010995.txt' replaced successfully.
File '006205.txt' replaced successfully.
File '011870.txt' replaced successfully.
File '016515.txt' replaced successfully.
File '005505.txt' replaced successfully.
File '001900.txt' replaced successfully.
File '013335.txt' replaced successfully.
File '029335.txt' replaced successfully.
File '014195.txt' replaced successfully.
File '029935.txt' replaced successfully.
File '017955.txt' replaced successfully.
File '025340.txt' replaced successfully.
File '025350.txt' replaced successfully.
File '011865.txt' replaced successfully.
File '029270.txt